In [1]:
!pip install nemo_toolkit['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.3 MB/s e

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
import os

source = '/gdrive/MyDrive/denoised_source'

files = os.listdir(source)
seasons = []
for file in files:
  if os.path.isdir(os.path.join(source, file)):
    seasons.append(file)
print(f"all seasons: {seasons}")
sub_dirs = []
for season in seasons:
  for f in os.listdir(os.path.join(source, season)):
    if(f.endswith('.wav')):
      base = os.path.basename(f)
      base, _ = os.path.splitext(base)
      sub_dirs.append(os.path.join(source, season, base))

print(sub_dirs[0])


all seasons: ['S3', 'S1', 'S2']
/gdrive/MyDrive/denoised_source/S3/EP1


In [6]:
import nemo.collections.asr as nemo_asr
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")


(…)peakerverification_en_titanet_large.nemo:   0%|          | 0.00/102M [00:00<?, ?B/s]

[NeMo W 2024-01-19 16:01:18 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-01-19 16:01:18 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-01-19 16:01:18 features:289] PADDING: 16
[NeMo I 2024-01-19 16:01:19 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/0dc382f40121a5fbd34db10a2bb04d826c2be6a8/speakerverification_en_titanet_large.nemo.


In [7]:
def clear_dir(path):
  base_name = os.path.basename(path)
  dir = os.path.dirname(path)
  for file in os.listdir(dir):
    if file != base_name:
      os.remove(os.path.join(dir, file))

In [8]:
target = os.path.join(source, "speaker_kaguya.wav")

In [9]:

for dir in sub_dirs:
  print(f'processing: {os.path.basename(dir)}')
  for speaker in os.listdir(dir):
    full_path = os.path.join(dir, speaker)
    if speaker_model.verify_speakers(full_path, target):
      print(f'{os.path.basename(full_path)} is the target')
      clear_dir(full_path)
      break


processing: EP1
[NeMo I 2024-01-19 16:01:39 label_models:585]  two audio files are from different speakers
[NeMo I 2024-01-19 16:01:41 label_models:582]  two audio files are from same speaker
speaker_1 is the target
processing: EP2
[NeMo I 2024-01-19 16:01:43 label_models:585]  two audio files are from different speakers
[NeMo I 2024-01-19 16:01:45 label_models:582]  two audio files are from same speaker
speaker_6 is the target
processing: EP3
[NeMo I 2024-01-19 16:01:47 label_models:585]  two audio files are from different speakers
[NeMo I 2024-01-19 16:01:48 label_models:585]  two audio files are from different speakers
[NeMo I 2024-01-19 16:01:50 label_models:582]  two audio files are from same speaker
speaker_2 is the target
processing: EP4
[NeMo I 2024-01-19 16:01:52 label_models:585]  two audio files are from different speakers
[NeMo I 2024-01-19 16:01:55 label_models:582]  two audio files are from same speaker
speaker_0 is the target
processing: EP5
[NeMo I 2024-01-19 16:01:57 l